# <ins> Week 3 Final </ins>
## Segmenting and Clustering Neighborhoods in Toronto

#### Importing needed libraries

In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen

#### Specifying the BeautifulSoup pre-requisites and scraping all data that lies within the "table" tags

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
table = soup.find_all("table")

#### Searching through the table "soup" for the tags tr and td (these hold the data want) and extracting that data into three np arrays: "postal_code," "borough," and "neighboorhood"


In [4]:
postal_code = []
borough = []
neighborhood = []

for table in table:
    rows = table.find_all('tr')
    
    for row in rows:
        cells = row.find_all('td')
        
        
        if len(cells) > 2:
            postal = cells[0]
            postal_code.append(postal.text.strip())
            
            bor = cells[1]
            borough.append(bor.text.strip())
            
            neigh = cells[2]
            neighborhood.append(neigh.text.strip())

#### Adding the three np arrays to a dataframe and naming the columns

In [5]:
df = pd.DataFrame(postal_code)
df.rename(columns={0:"PostalCode"}, inplace=True)
df['Borough'] = borough
df['Neighborhood'] = neighborhood
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


#### Deleting the rows where borough = "Not assigned" as well as extra rows pulled in by BeautifulSoup

In [6]:
df = df.drop(df[df.Borough == "Not assigned"].index)
df = df.drop(df[df.Borough == "NL"].index)
df = df.drop(df[df.Borough == "NS"].index)
df = df.drop(df[df.Borough == "B"].index)

df = df.reset_index(drop=True)

#### Code to replace  neighborhoods that are labelled as "Not assigned" with their borough name

In [7]:
df.loc[df['Neighborhood'] == "Not assigned", 'Neighborhood'] = df['Borough']
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Displays the df shape

In [8]:
df.shape

(103, 3)

#### Reading the geospatial data into a dataframe and merging it with the existing dataframe (df) using a left join

In [19]:
geocode = pd.read_csv('/Users/nsherfel/Desktop/Geospatial_Coordinates.csv')

df = df.merge(geocode, how='left')
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
